In [1]:
import pandas as pd
import numpy as np

In [2]:
base = 'https://api.collegefootballdata.com'

In [3]:
coach_df = pd.read_json('{}/coaches?minYear=2001'.format(base))
coach_df

,first_name,last_name,seasons
0,Steve,Addazio,"[{'school': 'Temple', 'year': 2011, 'games': 1..."
1,Terry,Allen,"[{'school': 'Kansas', 'year': 2001, 'games': 8..."
2,Tom,Allen,"[{'school': 'Indiana', 'year': 2016, 'games': ..."
3,Barry,Alvarez,"[{'school': 'Wisconsin', 'year': 2001, 'games'..."
4,Chuck,Amato,"[{'school': 'NC State', 'year': 2001, 'games':..."
...,...,...,...
415,Jeff,Woodruff,"[{'school': 'Eastern Michigan', 'year': 2001, ..."
416,Chris,Woods,"[{'school': 'Texas State', 'year': 2018, 'game..."
417,Brian,Wright,"[{'school': 'Florida Atlantic', 'year': 2013, ..."
418,Paul,Wulff,"[{'school': 'Washington State', 'year': 2008, ..."


In [4]:
coach_df2 = pd.DataFrame()
for index, row in coach_df.iterrows():
    temp_df = pd.json_normalize(coach_df['seasons'][index])
    temp_df['first_name'] = row['first_name']
    temp_df['last_name'] = row['last_name']
    if coach_df2.empty:
        coach_df2 = temp_df.copy()
    else:
        coach_df2 = pd.concat([coach_df2, temp_df])
coach_df2.sort_values(by=['year', 'school'], inplace=True)
coach_df2

,school,year,games,wins,losses,ties,preseason_rank,postseason_rank,srs,sp_overall,sp_offense,sp_defense,first_name,last_name
0,Air Force,2001,12,6,6,0,None,None,-10.7,-10.8,33.3,40.8,Fisher,DeBerry
0,Akron,2001,11,4,7,0,0,0,-9.7,-10,31.5,38.9,Lee,Owens
0,Alabama,2001,12,7,5,0,25,NaN,10.4,11.8,32.9,22.7,Dennis,Franchione
0,Arizona,2001,11,5,6,0,None,None,-4.2,-3.4,37.4,39.2,John,Mackovic
0,Arizona State,2001,11,4,7,0,NaN,NaN,2.1,2.2,40.8,39.1,Dirk,Koetter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,West Virginia,2020,0,0,0,0,None,None,2.3,8.5,27.9,19.4,Neal,Brown
1,Western Kentucky,2020,0,0,0,0,None,None,-9.4,-3.2,18.9,22.3,Tyson,Helton
3,Western Michigan,2020,0,0,0,0,None,None,None,None,None,None,Tim,Lester
8,Wisconsin,2020,0,0,0,0,12,NaN,23.8,26.5,38.7,12.2,Paul,Chryst


In [5]:
year_range = range(2001, 2021)
schedule_df = pd.DataFrame()
for year in year_range:
    temp_df = pd.read_json('{}/games?year={}'.format(base, str(year)))
    temp_df2 = pd.DataFrame({'year': year, 'week': pd.Series(temp_df['week'].unique())})
    if schedule_df.empty:
        schedule_df = temp_df2.copy()
    else:
        schedule_df = pd.concat([schedule_df, temp_df2])
schedule_df

,year,week
0,2001,1
1,2001,2
2,2001,3
3,2001,5
4,2001,6
...,...,...
11,2020,12
12,2020,13
13,2020,14
14,2020,15


In [6]:
results_df = pd.DataFrame()
for year in year_range:
    temp_df = pd.read_json('{}/games?year={}'.format(base, str(year)))
    temp_df2 = pd.merge(temp_df, coach_df2[coach_df2['year'] == year], how='left', left_on='home_team', right_on='school')
    temp_df2['home_coach'] = temp_df2['first_name'] + ' ' + temp_df2['last_name']
    temp_df2.rename(columns={"id": "game_id"}, inplace=True)
    temp_df3 = pd.merge(temp_df2, coach_df2[coach_df2['year'] == year], how='left', left_on='away_team', right_on='school')
    temp_df3['away_coach'] = temp_df3['first_name_y'] + ' ' + temp_df3['last_name_y']
    temp_df3 = temp_df3[['game_id','home_team','home_points','home_coach','away_team','away_points','away_coach']]
    temp_df3['year'] = year
    if results_df.empty:
        results_df = temp_df3.copy()
    else:
        results_df = pd.concat([results_df, temp_df3[~temp_df3['home_points'].isnull()]])
results_df['winning_team'] = np.where(results_df['home_points'] > results_df['away_points'], results_df['home_team'], results_df['away_team'])
results_df['losing_team'] = np.where(results_df['home_points'] > results_df['away_points'], results_df['away_team'], results_df['home_team'])
results_df

,game_id,home_team,home_points,home_coach,away_team,away_points,away_coach,year,winning_team,losing_team
0,212350097,Louisville,45.0,John Smith,New Mexico State,24.0,Tony Samuel,2001,Louisville,New Mexico State
1,63770,Nebraska,21.0,Frank Solich,TCU,7.0,Gary Patterson,2001,Nebraska,TCU
2,212370275,Wisconsin,26.0,Barry Alvarez,Virginia,17.0,Al Groh,2001,Wisconsin,Virginia
3,212370252,BYU,70.0,Gary Crowton,Tulane,35.0,Chris Scelfo,2001,BYU,Tulane
4,212370201,Oklahoma,10.0,Bob Stoops,North Carolina,0.0,John Bunting,2001,Oklahoma,North Carolina
...,...,...,...,...,...,...,...,...,...,...
518,401234572,Virginia Tech,33.0,Justin Fuente,Virginia,15.0,Bronco Mendenhall,2020,Virginia Tech,Virginia
519,401249432,Oregon State,24.0,Jonathan Smith,Stanford,27.0,David Shaw,2020,Stanford,Oregon State
520,401249435,BYU,28.0,Kalani Sitake,San Diego State,14.0,Brady Hoke,2020,BYU,San Diego State
521,401249036,New Mexico,49.0,Danny Gonzales,Fresno State,39.0,Kalen DeBoer,2020,New Mexico,Fresno State


In [27]:
score_change_df = pd.DataFrame()
print_year = '0'
for index, row in schedule_df.iterrows():
    year = str(row['year'])
    week = str(row['week'])
    if year != print_year:
        print_year = year
        print('Year: ' + print_year)
    temp_df = pd.read_json('{}/plays?year={}&week={}'.format(base, year, week))
    if len(temp_df.index) > 0:
        temp_df = temp_df[temp_df['scoring'] == True]
        temp_df = temp_df[['id','game_id','period','clock','offense','defense','offense_score','defense_score','scoring']].sort_values(by='id')
        if score_change_df.empty:
            score_change_df = temp_df.copy()
        else:
            score_change_df = pd.concat([score_change_df, temp_df])
score_change_df

Year: 2001
Year: 2002
Year: 2003
Year: 2004
Year: 2005
Year: 2006
Year: 2007
Year: 2008
Year: 2009
Year: 2010
Year: 2011
Year: 2012
Year: 2013
Year: 2014
Year: 2015
Year: 2016
Year: 2017
Year: 2018
Year: 2019
Year: 2020


,id,game_id,period,clock,offense,defense,offense_score,defense_score,scoring
0,2123702010001,212370201,1,"{'minutes': 15, 'seconds': 0}",North Carolina,Oklahoma,0,0,False
1,2123702010002,212370201,1,"{'minutes': 15, 'seconds': 0}",Oklahoma,North Carolina,0,0,False
2,2123702010101,212370201,1,"{'minutes': 14, 'seconds': 55}",North Carolina,Oklahoma,0,0,False
3,2123702010102,212370201,1,"{'minutes': 14, 'seconds': 55}",North Carolina,Oklahoma,0,0,False
4,2123702010103,212370201,1,"{'minutes': 14, 'seconds': 55}",North Carolina,Oklahoma,0,0,False
...,...,...,...,...,...,...,...,...,...
6653,401267164104989888,401267164,4,"{'minutes': 1, 'seconds': 0}",Arkansas,Alabama,3,52,False
6654,401267164104994880,401267164,4,"{'seconds': 50, 'minutes': 0}",Arkansas,Alabama,3,52,False
6655,401267164104996928,401267164,4,"{'seconds': 30, 'minutes': 0}",Arkansas,Alabama,3,52,False
6656,401267164104999808,401267164,4,"{'seconds': 1, 'minutes': 0}",Arkansas,Alabama,3,52,False


In [28]:
score_change_df['winning_team_temp'] = np.where(score_change_df['offense_score'] > score_change_df['defense_score'], score_change_df['offense'], np.where(score_change_df['offense_score'] < score_change_df['defense_score'], score_change_df['defense'], 'tie'))
score_change_df['losing_team_temp'] = np.where(score_change_df['offense_score'] > score_change_df['defense_score'], score_change_df['defense'], np.where(score_change_df['offense_score'] < score_change_df['defense_score'], score_change_df['offense'], 'tie'))
score_change_df['winning_team_score_temp'] = np.where(score_change_df['offense_score'] > score_change_df['defense_score'], score_change_df['offense_score'], score_change_df['defense_score'])
score_change_df['losing_team_score_temp'] = np.where(score_change_df['offense_score'] > score_change_df['defense_score'], score_change_df['defense_score'], score_change_df['offense_score'])
score_change_df.drop(columns=['offense','defense','offense_score','defense_score'], inplace=True)
score_change_df.to_csv('score_change.csv', index=False)
score_change_df

,id,game_id,period,clock,scoring,winning_team_temp,losing_team_temp,winning_team_score_temp,losing_team_score_temp
8,2123702010204,212370201,1,"{'minutes': 13, 'seconds': 33}",True,Oklahoma,North Carolina,3,0
35,2123800380409,212380038,1,"{'minutes': 6, 'seconds': 54}",True,Fresno State,Colorado,6,0
36,2123800380410,212380038,1,"{'minutes': 6, 'seconds': 54}",True,Fresno State,Colorado,7,0
48,2123800380703,212380038,1,"{'minutes': 3, 'seconds': 47}",True,Fresno State,Colorado,13,0
49,2123800380704,212380038,1,"{'minutes': 3, 'seconds': 47}",True,Fresno State,Colorado,14,0
...,...,...,...,...,...,...,...,...,...
6549,401267164102884928,401267164,2,"{'minutes': 11, 'seconds': 50}",True,Alabama,Arkansas,24,3
6564,401267164102935616,401267164,2,"{'minutes': 6, 'seconds': 43}",True,Alabama,Arkansas,31,3
6582,401267164102996608,401267164,2,"{'seconds': 33, 'minutes': 0}",True,Alabama,Arkansas,38,3
6603,401267164103927488,401267164,3,"{'minutes': 7, 'seconds': 24}",True,Alabama,Arkansas,45,3


In [29]:
score_change_df = pd.read_csv('score_change.csv')
score_change_df

,id,game_id,period,clock,scoring,winning_team_temp,losing_team_temp,winning_team_score_temp,losing_team_score_temp
0,2123702010204,212370201,1,"{'minutes': 13, 'seconds': 33}",True,Oklahoma,North Carolina,3,0
1,2123800380409,212380038,1,"{'minutes': 6, 'seconds': 54}",True,Fresno State,Colorado,6,0
2,2123800380410,212380038,1,"{'minutes': 6, 'seconds': 54}",True,Fresno State,Colorado,7,0
3,2123800380703,212380038,1,"{'minutes': 3, 'seconds': 47}",True,Fresno State,Colorado,13,0
4,2123800380704,212380038,1,"{'minutes': 3, 'seconds': 47}",True,Fresno State,Colorado,14,0
...,...,...,...,...,...,...,...,...,...
169779,401267164102884928,401267164,2,"{'minutes': 11, 'seconds': 50}",True,Alabama,Arkansas,24,3
169780,401267164102935616,401267164,2,"{'minutes': 6, 'seconds': 43}",True,Alabama,Arkansas,31,3
169781,401267164102996608,401267164,2,"{'seconds': 33, 'minutes': 0}",True,Alabama,Arkansas,38,3
169782,401267164103927488,401267164,3,"{'minutes': 7, 'seconds': 24}",True,Alabama,Arkansas,45,3


In [30]:
blowout_df = score_change_df[score_change_df['winning_team_score_temp'] - score_change_df['losing_team_score_temp'] >= 17].copy()
blowout_df

,id,game_id,period,clock,scoring,winning_team_temp,losing_team_temp,winning_team_score_temp,losing_team_score_temp
42,2124202182201,212420218,3,"{'minutes': 10, 'seconds': 55}",True,Temple,Navy,23,6
43,2124202182202,212420218,3,"{'minutes': 10, 'seconds': 55}",True,Temple,Navy,24,6
46,2124202182615,212420218,3,"{'minutes': 1, 'seconds': 58}",True,Temple,Navy,30,13
47,2124202182616,212420218,3,"{'minutes': 1, 'seconds': 58}",True,Temple,Navy,31,13
48,2124202183001,212420218,4,"{'minutes': 14, 'seconds': 50}",True,Temple,Navy,37,13
...,...,...,...,...,...,...,...,...,...
169779,401267164102884928,401267164,2,"{'minutes': 11, 'seconds': 50}",True,Alabama,Arkansas,24,3
169780,401267164102935616,401267164,2,"{'minutes': 6, 'seconds': 43}",True,Alabama,Arkansas,31,3
169781,401267164102996608,401267164,2,"{'seconds': 33, 'minutes': 0}",True,Alabama,Arkansas,38,3
169782,401267164103927488,401267164,3,"{'minutes': 7, 'seconds': 24}",True,Alabama,Arkansas,45,3


In [31]:
merged_df = pd.merge(results_df, blowout_df, on='game_id').sort_values(by='id')
# Ensure no plays are double-counted
merged_df.drop_duplicates(subset=['game_id','winning_team_score_temp','losing_team_score_temp'], inplace=True)
merged_df

,game_id,home_team,home_points,home_coach,away_team,away_points,away_coach,year,winning_team,losing_team,id,period,clock,scoring,winning_team_temp,losing_team_temp,winning_team_score_temp,losing_team_score_temp
0,212420218,Temple,45.0,Bobby Wallace,Navy,26.0,Charlie Weatherbie,2001,Temple,Navy,2124202182201,3,"{'minutes': 10, 'seconds': 55}",True,Temple,Navy,23,6
1,212420218,Temple,45.0,Bobby Wallace,Navy,26.0,Charlie Weatherbie,2001,Temple,Navy,2124202182202,3,"{'minutes': 10, 'seconds': 55}",True,Temple,Navy,24,6
2,212420218,Temple,45.0,Bobby Wallace,Navy,26.0,Charlie Weatherbie,2001,Temple,Navy,2124202182615,3,"{'minutes': 1, 'seconds': 58}",True,Temple,Navy,30,13
3,212420218,Temple,45.0,Bobby Wallace,Navy,26.0,Charlie Weatherbie,2001,Temple,Navy,2124202182616,3,"{'minutes': 1, 'seconds': 58}",True,Temple,Navy,31,13
4,212420218,Temple,45.0,Bobby Wallace,Navy,26.0,Charlie Weatherbie,2001,Temple,Navy,2124202183001,4,"{'minutes': 14, 'seconds': 50}",True,Temple,Navy,37,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53039,401267164,Arkansas,3.0,Sam Pittman,Alabama,52.0,Nick Saban,2020,Alabama,Arkansas,401267164102884928,2,"{'minutes': 11, 'seconds': 50}",True,Alabama,Arkansas,24,3
53040,401267164,Arkansas,3.0,Sam Pittman,Alabama,52.0,Nick Saban,2020,Alabama,Arkansas,401267164102935616,2,"{'minutes': 6, 'seconds': 43}",True,Alabama,Arkansas,31,3
53041,401267164,Arkansas,3.0,Sam Pittman,Alabama,52.0,Nick Saban,2020,Alabama,Arkansas,401267164102996608,2,"{'seconds': 33, 'minutes': 0}",True,Alabama,Arkansas,38,3
53042,401267164,Arkansas,3.0,Sam Pittman,Alabama,52.0,Nick Saban,2020,Alabama,Arkansas,401267164103927488,3,"{'minutes': 7, 'seconds': 24}",True,Alabama,Arkansas,45,3


In [32]:
blowout_games_df = merged_df.copy()
blowout_games_df['diff'] = blowout_games_df['winning_team_score_temp'] - blowout_games_df['losing_team_score_temp']
blowout_games_df = blowout_games_df.loc[blowout_games_df.groupby(by=['game_id','winning_team_temp'])['diff'].idxmax()].sort_values(by='id')
blowout_games_df

,game_id,home_team,home_points,home_coach,away_team,away_points,away_coach,year,winning_team,losing_team,id,period,clock,scoring,winning_team_temp,losing_team_temp,winning_team_score_temp,losing_team_score_temp,diff
8,212420218,Temple,45.0,Bobby Wallace,Navy,26.0,Charlie Weatherbie,2001,Temple,Navy,2124202183412,4,"{'minutes': 1, 'seconds': 56}",True,Temple,Navy,45,19,26
15,212422649,Toledo,38.0,Tom Amstutz,Minnesota,7.0,Glen Mason,2001,Toledo,Minnesota,2124226491507,3,"{'minutes': 12, 'seconds': 45}",True,Toledo,Minnesota,31,0,31
111,212440002,Auburn,30.0,Tommy Tuberville,Ball State,0.0,Bill Lynch,2001,Auburn,Ball State,2124400022804,4,"{'minutes': 9, 'seconds': 50}",True,Auburn,Ball State,30,0,30
73,212440025,California,17.0,Tom Holmoe,Illinois,44.0,Ron Turner,2001,Illinois,California,2124400251805,2,"{'minutes': 1, 'seconds': 5}",True,Illinois,California,38,7,31
139,212440030,USC,21.0,Pete Carroll,San José State,10.0,Fitz Hill,2001,USC,San José State,2124400302119,4,"{'minutes': 12, 'seconds': 25}",True,USC,San José State,21,3,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53097,401266711,TCU,52.0,Gary Patterson,Louisiana Tech,10.0,Skip Holtz,2020,TCU,Louisiana Tech,401266711103888000,3,"{'minutes': 11, 'seconds': 19}",True,TCU,Louisiana Tech,45,0,45
53020,401266789,Charlotte,19.0,Will Healy,Western Kentucky,37.0,Tyson Helton,2020,Western Kentucky,Charlotte,401266789104977728,4,"{'minutes': 2, 'seconds': 22}",True,Western Kentucky,Charlotte,37,13,24
53049,401267161,Missouri,14.0,Eli Drinkwitz,Georgia,49.0,Kirby Smart,2020,Georgia,Missouri,401267161104855296,4,"{'minutes': 14, 'seconds': 46}",True,Georgia,Missouri,49,14,35
53090,401267163,Vanderbilt,17.0,Derek Mason,Tennessee,42.0,Jeremy Pruitt,2020,Tennessee,Vanderbilt,401267163103874880,3,"{'minutes': 12, 'seconds': 50}",True,Tennessee,Vanderbilt,35,10,25


In [33]:
comeback_wins_df = blowout_games_df[blowout_games_df['winning_team'] != blowout_games_df['winning_team_temp']].copy()
comeback_wins_df

,game_id,home_team,home_points,home_coach,away_team,away_points,away_coach,year,winning_team,losing_team,id,period,clock,scoring,winning_team_temp,losing_team_temp,winning_team_score_temp,losing_team_score_temp,diff
937,212860194,Ohio State,17.0,Jim Tressel,Wisconsin,20.0,Barry Alvarez,2001,Wisconsin,Ohio State,2128601940712,2,"{'minutes': 10, 'seconds': 32}",True,Ohio State,Wisconsin,17,0,17
1011,213000213,Penn State,29.0,Joe Paterno,Ohio State,27.0,Jim Tressel,2001,Penn State,Ohio State,2130002132503,3,"{'minutes': 13, 'seconds': 54}",True,Ohio State,Penn State,27,9,18
1318,213140153,North Carolina,31.0,John Bunting,Wake Forest,32.0,Jim Grobe,2001,Wake Forest,North Carolina,2131401531408,2,"{'minutes': 2, 'seconds': 20}",True,North Carolina,Wake Forest,24,0,24
1529,213210096,Kentucky,35.0,Guy Morriss,Tennessee,38.0,Phillip Fulmer,2001,Tennessee,Kentucky,2132100961109,2,"{'minutes': 5, 'seconds': 39}",True,Kentucky,Tennessee,21,0,21
1714,213280127,Michigan State,37.0,Bobby Williams,Penn State,42.0,Joe Paterno,2001,Penn State,Michigan State,2132801271209,2,"{'minutes': 10, 'seconds': 24}",True,Michigan State,Penn State,24,7,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52778,401247324,Rutgers,42.0,Greg Schiano,Michigan,48.0,Jim Harbaugh,2020,Michigan,Rutgers,401247324102957632,2,"{'minutes': 4, 'seconds': 23}",True,Rutgers,Michigan,17,0,17
52404,401249003,New Mexico,21.0,Danny Gonzales,San José State,38.0,Brent Brennan,2020,San José State,New Mexico,401249003104905984,4,"{'minutes': 9, 'seconds': 39}",True,New Mexico,San José State,38,21,17
52984,401249033,San José State,35.0,Brent Brennan,Hawai'i,24.0,Todd Graham,2020,San José State,Hawai'i,401249033101937472,1,"{'minutes': 6, 'seconds': 24}",True,Hawai'i,San José State,21,0,21
53101,401249431,UCLA,38.0,Chip Kelly,USC,43.0,Clay Helton,2020,USC,UCLA,401249431103877120,3,"{'minutes': 12, 'seconds': 28}",True,UCLA,USC,28,10,18


In [34]:
double_comeback_games_df = blowout_games_df[blowout_games_df.duplicated(subset='game_id', keep=False)].copy()
double_comeback_games_df

,game_id,home_team,home_points,home_coach,away_team,away_points,away_coach,year,winning_team,losing_team,id,period,clock,scoring,winning_team_temp,losing_team_temp,winning_team_score_temp,losing_team_score_temp,diff
3403,232980096,Kentucky,42.0,Rich Brooks,Mississippi State,17.0,Jackie Sherrill,2003,Kentucky,Mississippi State,2329800961108,2,"{'minutes': 12, 'seconds': 10}",True,Mississippi State,Kentucky,17,0,17
3407,232980096,Kentucky,42.0,Rich Brooks,Mississippi State,17.0,Jackie Sherrill,2003,Kentucky,Mississippi State,2329800963510,4,"{'minutes': 2, 'seconds': 49}",True,Kentucky,Mississippi State,42,17,25
4820,242622641,Texas Tech,70.0,Mike Leach,TCU,35.0,Gary Patterson,2004,Texas Tech,TCU,2426226411608,2,"{'minutes': 8, 'seconds': 10}",True,TCU,Texas Tech,21,0,21
4826,242622641,Texas Tech,70.0,Mike Leach,TCU,35.0,Gary Patterson,2004,Texas Tech,TCU,2426226414204,4,"{'minutes': 13, 'seconds': 12}",True,Texas Tech,TCU,56,21,35
5824,243110251,Texas,56.0,Mack Brown,Oklahoma State,35.0,Les Miles,2004,Texas,Oklahoma State,2431102511613,2,"{'minutes': 1, 'seconds': 18}",True,Oklahoma State,Texas,35,7,28
5828,243110251,Texas,56.0,Mack Brown,Oklahoma State,35.0,Les Miles,2004,Texas,Oklahoma State,2431102513703,4,"{'minutes': 7, 'seconds': 22}",True,Texas,Oklahoma State,56,35,21
7634,252872655,Tulane,21.0,Chris Scelfo,UTEP,45.0,Mike Price,2005,UTEP,Tulane,252872655150,3,"{'minutes': 8, 'seconds': 7}",True,Tulane,UTEP,21,0,21
7637,252872655,Tulane,21.0,Chris Scelfo,UTEP,45.0,Mike Price,2005,UTEP,Tulane,252872655202,4,"{'minutes': 10, 'seconds': 34}",True,UTEP,Tulane,45,21,24
8069,253020197,Oklahoma State,28.0,Mike Gundy,Texas,47.0,Mack Brown,2005,Texas,Oklahoma State,253020197091,2,"{'minutes': 5, 'seconds': 44}",True,Oklahoma State,Texas,28,9,19
8071,253020197,Oklahoma State,28.0,Mike Gundy,Texas,47.0,Mack Brown,2005,Texas,Oklahoma State,253020197205,4,"{'minutes': 3, 'seconds': 39}",True,Texas,Oklahoma State,47,28,19
